In [1]:
# import libraries 
import pandas as pd
import numpy as np
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import metrics


import geopy
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

from vincenty import vincenty

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

from sklearn.cluster import KMeans

from imblearn.over_sampling import SMOTE


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from sklearn.svm import SVC
import sklearn.metrics
    
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import model_selection


C:\Users\erkin\miniconda3\envs\PDS20\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("../data/processed/dortmund_trips.csv", index_col = 0)

In [5]:
df.describe()

,b_number,latitude_start,longitude_start,latitude_end,longitude_end,trip_duration,distance,weekday,weekend,day,month,hour,temperature °C,precipitation in mm,precipitation,area_start
count,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000,207476.000000
mean,206360.600031,51.507129,7.456074,51.507193,7.456184,33.376010,0.907952,2.758546,0.216011,16.015317,7.187882,12.556942,12.309225,0.055196,0.112066,2.915532
std,213713.119581,0.011740,0.025233,0.011730,0.025143,83.413993,1.160495,1.915539,0.411523,8.623309,3.286835,5.812148,7.284724,0.298039,0.315448,1.021359
min,843.000000,51.476038,7.334661,51.476038,7.334661,2.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,-8.900000,0.000000,0.000000,1.000000
25%,50846.000000,51.498563,7.444455,51.498845,7.444455,3.000000,0.000000,1.000000,0.000000,9.000000,4.000000,8.000000,6.700000,0.000000,0.000000,2.000000
50%,53006.000000,51.509557,7.459931,51.509557,7.459931,9.000000,0.632603,3.000000,0.000000,16.000000,8.000000,13.000000,11.700000,0.000000,0.000000,3.000000
75%,500100.000000,51.516234,7.469189,51.516234,7.469189,21.000000,1.339151,4.000000,0.000000,24.000000,10.000000,17.000000,17.200000,0.000000,0.000000,4.000000
max,500864.000000,51.541818,7.557450,51.541818,7.557450,1399.000000,10.505409,6.000000,1.000000,31.000000,12.000000,23.000000,35.000000,7.700000,1.000000,4.000000


In [12]:
def make_point(row):
    return Point(row.longitude_start, row.latitude_start)


In [13]:
#these are currently the coordinates of TU Dortmund Hörsaalgebäude 2
def calculate_distanceToUniversity(row):
    # mean of the university-station-coordinates
    university_center_lat = (51.492296 + 51.491721 + 51.49269 + 51.493966 + 51.493695) / 5
    university_center_lon = (7.41273 + 7.409468 + 7.417633 + 7.418008 + 7.420396) / 5
    
    # distance of the start station to the "center" of the university-area
    distance = vincenty([row["latitude_start"], row["longitude_start"]], [university_center_lat, university_center_lon],)
    
    return distance

In [14]:
def get_tripLabel(row):
    if ((row['towardsUniversity'] == 1) & (row['awayFromUniversity'] == 0)):
        return 'towardsUniversity'
    if ((row['towardsUniversity'] == 0) & (row['awayFromUniversity'] == 1)):
        return 'awayFromUniveristy'
    if ((row['towardsUniversity'] == 1) & (row['awayFromUniversity'] == 1)):
        return 'towardsUniversity'
    if ((row['towardsUniversity'] == 0) & (row['awayFromUniversity'] == 0)):
        return 'noUniversityRide'
    
    warnings.warn("Warning...........Message")
    return None

In [15]:
def prepare_data(df):

    # ToDo: Diesen geometry-step könnte man auch bei preprocessing reinpacken, dann müssen wir das nicht doppelt berechnen
    
    # Go through every row, and make a point out of its lat and lon
    df["geometry"] = df.apply(make_point, axis=1)
    # It doesn't come with a CRS because it's a CSV, so it has to be set
    df.crs = {'init': 'epsg:4326'}
    
    # get geodata of germany (postal codes and their areas/polygons)
    districts_germany = gpd.read_file("../data/external/germany_postalcodes.geojson")
    # filter for districts of dortmund
    districts_dortmund = districts_germany[districts_germany["note"].str.contains("Dortmund")]
    
    #convert dataset of trips to geodataframe (so it can be merged later with the geodataframe of dortmund)
    geo_df = gpd.GeoDataFrame(df, crs={'init': 'epsg:4326'}, geometry=df.geometry)
    
    # join the data
    # merges data when POINT of trips is within POLYGON of a dortmund district
    df_merged = gpd.sjoin(geo_df, districts_dortmund, how='left', op='within')
    
    # adding the distance between start position and the center of the university
    df_merged["distanceToUniversity"] = df_merged.apply(calculate_distanceToUniversity,axis=1)

    
    # add the attribute whether a trip was done towars/away from university
    university_stations = ["TU Dortmund Seminarraumgebäude 1", "TU Dortmund Hörsaalgebäude 2", "Universität/S-Bahnhof", "TU Dortmund Emil-Figge-Straße 50", "FH-Dortmund Emil-Figge-Straße 42"]

    df_merged['towardsUniversity'] = df_merged['p_name_end'].apply(lambda x: 1 if x in university_stations else 0)
    df_merged['awayFromUniversity'] = df_merged['p_name_start'].apply(lambda x: 1 if x in university_stations else 0)
    
    df_merged['tripLabel'] = df_merged.apply(lambda row: get_tripLabel(row), axis=1)
    
    #df_merged['area_start'] = ... # if we receive input data we have to calculate the attribute 'area_start'
    # here I don't do it, because it already exists

    
    return df_merged

In [16]:
data = prepare_data(df)

In [22]:
# this was done so I could access the data fast
# data.to_csv('../data/processed/prediction_data_with_outliers.csv')

In [3]:
data = pd.read_csv('../data/processed/prediction_data_with_outliers.csv', index_col = 0)


In [17]:
data

,datetime_start,b_number,latitude_start,p_name_start,longitude_start,datetime_end,latitude_end,p_name_end,longitude_end,trip_duration,...,geometry,index_right,plz,note,qkm,einwohner,distanceToUniversity,towardsUniversity,awayFromUniversity,tripLabel
0,2019-01-20 16:22:00,50641,51.506312,Hainallee / Südbad,7.470531,2019-01-20 17:00:00,51.493966,TU Dortmund Emil-Figge-Straße 50,7.418008,38,...,POINT (7.47053 51.50631),3073,44139,44139 Dortmund,4.896154,19843,4.094004,1,0,towardsUniversity
1,2019-01-20 16:42:00,53940,51.507457,Möllerbrücke,7.451364,2019-01-20 16:44:00,51.507457,Möllerbrücke,7.451364,2,...,POINT (7.45136 51.50746),3072,44137,44137 Dortmund,3.281205,21573,2.963796,0,0,noUniversityRide
2,2019-01-20 16:53:00,50061,51.503293,Vinckeplatz,7.455822,2019-01-20 17:13:00,51.519332,Cinestar,7.460124,20,...,POINT (7.45582 51.50329),3073,44139,44139 Dortmund,4.896154,19843,3.021170,0,0,noUniversityRide
3,2019-01-20 16:35:00,51138,51.499039,Steigenberger Hotel / Berswordtstr.,7.451472,2019-01-20 16:37:00,51.499039,Steigenberger Hotel / Berswordtstr.,7.451472,2,...,POINT (7.45147 51.49904),3073,44139,44139 Dortmund,4.896154,19843,2.580733,0,0,noUniversityRide
4,2019-01-20 16:43:00,53120,51.507457,Möllerbrücke,7.451364,2019-01-20 17:02:00,51.512836,Am Kaiserbrunnen,7.482258,19,...,POINT (7.45136 51.50746),3072,44137,44137 Dortmund,3.281205,21573,2.963796,0,0,noUniversityRide
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207471,2019-12-31 09:10:00,500084,51.532685,Immermannstr./Klinikzentrum,7.454631,2019-12-31 09:21:00,51.516831,Kuckelke,7.469189,11,...,POINT (7.45463 51.53269),3077,44147,44147 Dortmund,8.196036,22692,5.190721,0,0,noUniversityRide
207472,2019-12-31 09:07:00,500011,51.532685,Immermannstr./Klinikzentrum,7.454631,2019-12-31 09:21:00,51.516831,Kuckelke,7.469189,14,...,POINT (7.45463 51.53269),3077,44147,44147 Dortmund,8.196036,22692,5.190721,0,0,noUniversityRide
207473,2019-12-31 09:11:00,500837,51.492690,Universität/S-Bahnhof,7.417633,2019-12-31 09:31:00,51.512609,Hansastr.,7.463483,20,...,POINT (7.41763 51.49269),3080,44227,44227 Dortmund,19.106338,17922,0.139429,0,1,awayFromUniveristy
207474,2019-12-31 09:57:00,500118,51.518297,Geschwister-Scholl-Str.,7.474962,2019-12-31 10:08:00,51.512609,Hansastr.,7.463483,11,...,POINT (7.47496 51.51830),3071,44135,44135 Dortmund,1.469342,11921,4.996192,0,0,noUniversityRide


## LogisticRegression

### In the following the prediction for the attributes towards- and awayFromUniversity was done separately 

In [15]:
# How to optimize hyper-parameters of a Logistic Regression model using Grid Search in Python
def optimize_hyperparameters(X,y):

    # Create an scaler object
    sc = StandardScaler()

    # Create a pca object
    pca = decomposition.PCA()

    # Create a logistic regression object with an L2 penalty
    logistic = linear_model.LogisticRegression()

    # Create a pipeline of three steps. First, standardize the data.
    # Second, tranform the data with PCA.
    # Third, train a logistic regression on the data.
    pipe = Pipeline(steps=[('sc', sc),
                           ('pca', pca),
                           ('logistic', logistic)])

    # Create Parameter Space
    # Create a list of a sequence of integers from 1 to 30 (the number of features in X + 1)
    n_components = list(range(1,X.shape[1]+1,1))
    # Create a list of values of the regularization parameter
    C = np.logspace(-4, 4, 50)
    # Create a list of options for the regularization penalty
    penalty = ['l1', 'l2']
    # Create a dictionary of all the parameter options 
    # Note has you can access the parameters of steps of a pipeline by using '__’
    parameters = dict(pca__n_components=n_components,
                      logistic__C=C,
                      logistic__penalty=penalty)

    # Conduct Parameter Optmization With Pipeline
    # Create a grid search object
    clf = GridSearchCV(pipe, parameters)

    # Fit the grid search
    clf.fit(X, y)
    # View The Best Parameters
    print('Best Penalty:', clf.best_estimator_.get_params()['logistic__penalty'])
    print('Best C:', clf.best_estimator_.get_params()['logistic__C'])
    print('Best Number Of Components:', clf.best_estimator_.get_params()['pca__n_components'])
    print(); print(clf.best_estimator_.get_params()['logistic'])

    # Use Cross Validation To Evaluate Model
    CV_Result = cross_val_score(clf, X, y, cv=4, n_jobs=-1)
    print(); print(CV_Result)
    print(); print(CV_Result.mean())
    print(); print(CV_Result.std())
    


In [18]:
y_towards = data['towardsUniversity']
y_away = data['awayFromUniversity']

# use only start-information to classify the trip-class
X = data[['weekend', 'hour', 'distanceToUniversity', 'month', 'area_start']]

### These are the best parameters for logistic regression for the attribute awayFromUniversity

In [18]:
optimize_hyperparameters(X,y_away)

Best Penalty: l2
Best C: 0.12648552168552957
Best Number Of Components: 4

LogisticRegression(C=0.12648552168552957, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

[0.9854441  0.98681293 0.98557905 0.98640807]

0.9860610383851627

0.0005698014886013382


### These are the best parameters for logistic regression for the attribute towardsUniversity

In [19]:
optimize_hyperparameters(X,y_towards)

Best Penalty: l2
Best C: 0.8286427728546842
Best Number Of Components: 5

LogisticRegression(C=0.8286427728546842, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

[0.91808209 0.91808209 0.91684821 0.91573001]

0.9171856021901329

0.0009797906933469204


#### Performance of the models

In [6]:
from sklearn.decomposition import PCA

pca_model = PCA(n_components=4).fit(X)
X_pca = pca_model.transform(X)

In [7]:
# create model based on the optimal parameters

X_train, X_test, y_train, y_test = train_test_split(X_pca, y_away, test_size=0.3)
st_scaler = StandardScaler()
X_train_scaled = st_scaler.fit_transform(X_train)


log = LogisticRegression(C=0.12648552168552957, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
log.fit(X_train_scaled, y_train)

X_test_scaled = st_scaler.transform(X_test)
y_predict = log.predict(X_test_scaled)


print(classification_report(y_true=y_test, y_pred=y_predict))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     56941
           1       0.87      0.99      0.93      5302

    accuracy                           0.99     62243
   macro avg       0.93      0.99      0.96     62243
weighted avg       0.99      0.99      0.99     62243



In [2]:
# MODEL PERFORMACE FOR AWAYFROMUNIVERSITY

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y_towards, test_size=0.3)
st_scaler = StandardScaler()
X_train_scaled = st_scaler.fit_transform(X_train)


log = LogisticRegression(C=0.8286427728546842, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
log.fit(X_train_scaled, y_train)

X_test_scaled = st_scaler.transform(X_test)
y_predict = log.predict(X_test_scaled)


print(classification_report(y_true=y_test, y_pred=y_predict))

              precision    recall  f1-score   support

           0       0.92      0.99      0.96     57109
           1       0.53      0.08      0.14      5134

    accuracy                           0.92     62243
   macro avg       0.73      0.54      0.55     62243
weighted avg       0.89      0.92      0.89     62243



OS DOES NOT WORK FOR OptimizeParameters because it has to be done on the training set (and not on the test set), but optimizeparameters processes both

In [ ]:
# # OS Für towardsUNI
# X_train, X_test, y_train, y_test = train_test_split(X, y_towards, test_size=0.3)
# os = SMOTE(random_state=0)

# columns = X_train.columns
# X_train_res,y_train_res =os.fit_sample(X_train, y_train.ravel())


# print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))
# print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))

# st_scaler = StandardScaler()
# X_train_res_scaled = st_scaler.fit_transform(X_train_res)


# log = LogisticRegression(C=4.0, multi_class='multinomial', solver = 'newton-cg')
# log.fit(X_train_res_scaled, y_train_res)

# X_test_scaled = st_scaler.transform(X_test)
# y_predict = log.predict(X_test_scaled)

# print(classification_report(y_true=y_test, y_pred=y_predict))

In [ ]:
def predict_awayFromUniversity(X):
    # set the model
    X_train, X_test, y_train, y_test = train_test_split(X, y_towards, test_size=0.3)
    st_scaler = StandardScaler()
    X_train_scaled = st_scaler.fit_transform(X_train)


    # TODO: Modell anpassen
    log = LogisticRegression(C=0.18420699693267145, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
    log.fit(X_train_scaled, y_train)
    
    # scale the input
    X_scaled = st_scaler.fit_transform(X)
    
    # predict outcome
    y_pred = log.predict(X_scaled)
    
    return y_pred

## SVM

I had to stop this function, because it lasted too long (stopped after about 1 day)

In [11]:
def optimize_hyperparameters_svm(X,y, nfolds):
    # defining parameter range 
    param_grid = [{'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
                  {'C': [0.1, 1, 10, 100, 1000],  
                  'gamma': [1, 0.5, 0.3, 0.1, 0.01, 0.001, 0.0001], 
                  'kernel': ['rbf']}]

    grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, cv=nfolds) 

    # fitting the model for grid search 
    grid.fit(X, y) 
    
    # print best parameter after tuning 
    print(grid.best_params_) 
  
    # print how our model looks after hyper-parameter tuning 
    print(grid.best_estimator_)

In [ ]:
optimize_hyperparameters_svm(X,y_away, 5)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ C=0.1, kernel=linear, score=0.977, total=  49.2s
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   49.3s remaining:    0.0s


[CV] ................ C=0.1, kernel=linear, score=0.978, total= 3.1min
[CV] C=0.1, kernel=linear ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.9min remaining:    0.0s


[CV] ................ C=0.1, kernel=linear, score=0.982, total= 1.5min
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.982, total=  52.8s
[CV] C=0.1, kernel=linear ............................................
[CV] ................ C=0.1, kernel=linear, score=0.979, total= 1.3min
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.977, total= 2.1min
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.978, total= 2.3min
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.976, total= 2.1min
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.982, total= 1.9min
[CV] C=1, kernel=linear ..............................................
[CV] .

[CV] ............ C=1, gamma=1, kernel=rbf, score=0.923, total= 9.1min
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.979, total= 9.7min
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.955, total= 9.6min
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.979, total=11.3min
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ............ C=1, gamma=1, kernel=rbf, score=0.946, total=11.3min
[CV] C=1, gamma=0.5, kernel=rbf ......................................
[CV] .......... C=1, gamma=0.5, kernel=rbf, score=0.946, total= 2.5min
[CV] C=1, gamma=0.5, kernel=rbf ......................................
[CV] .......... C=1, gamma=0.5, kernel=rbf, score=0.978, total= 6.2min
[CV] C=1, gamma=0.5, kernel=rbf ......................................
[CV] .

[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.984, total=  50.3s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.979, total=  51.4s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.977, total=  51.1s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.978, total=  54.8s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.982, total=  54.5s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.982, total=  55.2s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.979, total=  56.7s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] .

# Random Forrest which does not separate between the attributes
## This model predict the categorical variable "TripLabel" which takes the values "awayFromUniversity", "towardsUniversity" and "noUniversityRide" 

In [9]:
def optimize_hyperparameters_randomforest(X,y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    # random forest model creation
    rfc = RandomForestClassifier()
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}

    # Random search of parameters
    rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the model
    rfc_random.fit(X_train, y_train)
    # print results
    print(rfc_random.best_params_)

In [20]:
y = data['tripLabel']

# use only start-information to classify the trip-class
X = data[['weekend', 'hour', 'distanceToUniversity', 'month', 'area_start']]

In [11]:
optimize_hyperparameters_randomforest(X,y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 36.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 174.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 339.9min finished


{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False}


### Performance

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# ANPASSEN !!!! basierend auf den Ergebnissen aus Zeile vorher
rfc = RandomForestClassifier(n_estimators=1000, min_samples_split = 2,min_samples_leaf=1, max_depth=10, max_features='auto', bootstrap= False)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, y, cv=10)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[ 2527     1   527]
 [    6 53960   150]
 [ 1086  2670  1316]]


=== Classification Report ===
                    precision    recall  f1-score   support

awayFromUniveristy       0.70      0.83      0.76      3055
  noUniversityRide       0.95      1.00      0.97     54116
 towardsUniversity       0.66      0.26      0.37      5072

          accuracy                           0.93     62243
         macro avg       0.77      0.69      0.70     62243
      weighted avg       0.92      0.93      0.91     62243



=== All AUC Scores ===
[0.28142472 0.88143436 0.88167534 0.87516869 0.88018122 0.91517255
 0.91256567 0.91169808 0.90596231 0.91748205]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8362764975327857


## RandomForest
### Prediction model for both attributes separately


### AwayFromUniversity

In [12]:
y_away = data['awayFromUniversity']

In [13]:
optimize_hyperparameters_randomforest(X,y_away)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 303.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 340.8min finished


{'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True}


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y_away, test_size=0.3)

# ANPASSEN !!!! basierend auf den Ergebnissen aus Zeile vorher
rfc = RandomForestClassifier(n_estimators=400, min_samples_split = 5,min_samples_leaf=1, max_depth=30, max_features='sqrt', bootstrap= True)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, y_away, cv=10, scoring='roc_auc')

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[56975     0]
 [    0  5268]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56975
           1       1.00      1.00      1.00      5268

    accuracy                           1.00     62243
   macro avg       1.00      1.00      1.00     62243
weighted avg       1.00      1.00      1.00     62243



=== All AUC Scores ===
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  1.0


### towardsUniversity

In [14]:
y_towards = data['towardsUniversity']

In [15]:
optimize_hyperparameters_randomforest(X,y_towards)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 32.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 152.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 293.0min finished


{'n_estimators': 2000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y_towards, test_size=0.3)

rfc = RandomForestClassifier(n_estimators=2000, min_samples_split = 5,min_samples_leaf=1, max_depth=10, max_features='sqrt', bootstrap= True)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, y_towards, cv=10, scoring='roc_auc')

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[56783   459]
 [ 3951  1050]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     57242
           1       0.70      0.21      0.32      5001

    accuracy                           0.93     62243
   macro avg       0.82      0.60      0.64     62243
weighted avg       0.92      0.93      0.91     62243



=== All AUC Scores ===
[0.62230677 0.89718263 0.89805715 0.87987842 0.89694665 0.8827083
 0.87964691 0.86347401 0.8492312  0.88101501]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8550447057307388


## Prediction Process:

Assumption: We get a dataset/dataframe X with trips that have to be predicted

1) Define the KMeans-clustering variable km

2) Define the 2 models (pred_model_away, pred_model_towards)

3) get the clustered start area : cluster_area_start(X, km)

4) calculate the relevant attributes for the prediction: preprocess(X)

5) Predict the attributes 'towardsUniversity' and 'awayFromUniversity': predict_towardsUniversity(X), predict_awayFromUniversity(X)

6) Get the output-code. This means the predicted values 0/1 are mapped to the labels "towardsUniversty", "awayFromUniversity", "noUniversityRide"